### Lab 4 Task 1
#### Обраний текст: https://www.kaggle.com/datasets/mykras/ukrainian-texts, Лис Микита

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from collections import Counter
import random
import numpy as np
import re

# Фіксація випадкових чисел для відтворюваності
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# Завантаження тексту з файлу
filename = 'Lys_mykyta.txt'
try:
    with open(filename, 'r', encoding='utf-8') as f:
        text_raw = f.read()
    print(f"Файл {filename} завантажено. Довжина тексту - {len(text_raw)} символів")
except FileNotFoundError:
    print(f"Файл {filename} не знайдено.")
    text_raw = "" 

def clean_text(text):
    text = text.lower()
    # Залишаємо букви, цифри, апостроф, дефіс та пунктуацію
    # \u2019 - це типографський апостроф, ' - звичайний
    text = re.sub(r"[^а-яіїєґa-z0-9\s\-\.,!?'\u2019]", "", text) 
    text = re.sub(r"\s+", " ", text)
    return text.strip()

corpus = clean_text(text_raw)

# Токенізація на слова та знаки пунктуації
def tokenize(text):
    # Цей regex шукає:
    # 1. Слова з апострофами та дефісами (м'ясо, по-нашому)
    # 2. АБО окремі розділові знаки
    pattern = r"[а-яіїєґa-z0-9]+(?:['\u2019\-][а-яіїєґa-z0-9]+)*|[.,!?;]"
    return re.findall(pattern, text)

tokens = tokenize(corpus)

MAX_VOCAB = 5000
specials = ["[PAD]", "[UNK]", "[BOS]", "[EOS]"]

counter = Counter(tokens)
most_common = counter.most_common(MAX_VOCAB - len(specials))
itos = specials + [w for w, _ in most_common]
stoi = {w: i for i, w in enumerate(itos)}

PAD_IDX = stoi["[PAD]"]
UNK_IDX = stoi["[UNK]"]
vocab_size = len(stoi)

print(f"Vocab size: {vocab_size}")

BLOCK_SIZE = 16

# Підготовка даних для навчання
class TextGenerationDataset(Dataset):
    def __init__(self, tokens, stoi, block_size):
        self.data = [stoi.get(t, UNK_IDX) for t in tokens]
        self.block_size = block_size
        
    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        chunk = self.data[idx : idx + self.block_size + 1]
        chunk_tensor = torch.tensor(chunk, dtype=torch.long)
        return chunk_tensor[:-1], chunk_tensor[1:]

dataset = TextGenerationDataset(tokens, stoi, BLOCK_SIZE)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, drop_last=True)

# Створення маски щоб приховати майбутні слова
def generate_subsequent_mask(size, device):
    mask = torch.triu(torch.ones(size, size, dtype=torch.bool, device=device), diagonal=1)
    return mask

class TokenPositionalEmbedding(nn.Module):
    def __init__(self, vocab_size, d_model, max_len=100, dropout=0.1):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, d_model, padding_idx=PAD_IDX)
        self.pos_emb = nn.Embedding(max_len, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T = x.shape
        pos = torch.arange(T, device=x.device).unsqueeze(0).expand(B, T)
        x = self.token_emb(x) + self.pos_emb(pos)
        return self.dropout(x)

class TransformerGenerator(nn.Module):
    def __init__(self, vocab_size, d_model=64, n_heads=4, num_layers=2, d_ff=256, dropout=0.2):
        super().__init__()
        self.embedding = TokenPositionalEmbedding(vocab_size, d_model, max_len=BLOCK_SIZE, dropout=dropout)
        
        dec_layer = nn.TransformerDecoderLayer(
            d_model=d_model, 
            nhead=n_heads, 
            dim_feedforward=d_ff, 
            dropout=dropout, 
            batch_first=True
        )
        self.decoder = nn.TransformerDecoder(dec_layer, num_layers=num_layers)
        self.output_proj = nn.Linear(d_model, vocab_size)

    def forward(self, src):
        x = self.embedding(src)
        
        T = src.size(1)
        mask = generate_subsequent_mask(T, src.device)
        
        out = self.decoder(
            tgt=x, 
            memory=x, 
            tgt_mask=mask
        )
        
        logits = self.output_proj(out)
        return logits

model = TransformerGenerator(vocab_size=vocab_size).to(device)
print(f"Parameters: {sum(p.numel() for p in model.parameters())}")

# Навчання моделі
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

n_epochs = 30 
print("Початок навчання")

for epoch in range(n_epochs):
    model.train()
    total_loss = 0
    
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        
        optimizer.zero_grad()
        logits = model(x)
        
        B, T, V = logits.shape
        loss = criterion(logits.view(B*T, V), y.view(B*T))
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1} | Loss: {total_loss/len(dataloader):.4f}")

# Функція генерації тексту
@torch.no_grad()
def generate_text(model, start_text, max_new_tokens=50, temperature=1.0, top_k=10, repetition_penalty=1.2):
    model.eval()
    
    tokens = tokenize(clean_text(start_text))
    idx = [stoi.get(t, UNK_IDX) for t in tokens]
    
    # Захист від порожнього входу
    if not idx:
        idx = [stoi.get(tokens[0], UNK_IDX)] if tokens else [UNK_IDX]
        
    input_tensor = torch.tensor(idx, dtype=torch.long, device=device).unsqueeze(0)
    
    result_idx = idx[:]

    for _ in range(max_new_tokens):
        cond = input_tensor[:, -BLOCK_SIZE:]
        
        logits = model(cond)
        last_logits = logits[:, -1, :] 
        
        for token_id in set(result_idx):
            if last_logits[0, token_id] < 0:
                last_logits[0, token_id] *= repetition_penalty
            else:
                last_logits[0, token_id] /= repetition_penalty

        last_logits = last_logits / temperature

        if top_k is not None:
            v, _ = torch.topk(last_logits, top_k)
            last_logits[last_logits < v[:, [-1]]] = -float('Inf')

        probs = torch.softmax(last_logits, dim=-1)
        
        # Випадковий вибір зваженого токена
        next_idx = torch.multinomial(probs, num_samples=1).item()
        
        result_idx.append(next_idx)
        input_tensor = torch.cat((input_tensor, torch.tensor([[next_idx]], device=device)), dim=1)

        if next_idx == stoi.get("[EOS]", -1):
            break

    decoded = [itos[i] for i in result_idx]
    output_text = " ".join(decoded)
    output_text = re.sub(r'\s([.,!?;])', r'\1', output_text)
    
    return output_text

print("\nПриклади генерації")
print("1 Temp 0.5:", generate_text(model, "Лис Микита біг", temperature=0.5, repetition_penalty=1))
print("2 Temp 0.7:", generate_text(model, "Цар Лев", temperature=0.7, repetition_penalty=1.5))
print("3 Temp 1.0:", generate_text(model, "Ті безхвості", temperature=1.0, repetition_penalty=1.7))
print("4 Temp 1.2:", generate_text(model, "Вийшли в поле", temperature=1.2, repetition_penalty=2))

Device: cuda
Файл Lys_mykyta.txt завантажено. Довжина тексту - 86526 символів
Vocab size: 5000
Parameters: 779528
Початок навчання
Epoch 10 | Loss: 0.3860
Epoch 20 | Loss: 0.3097
Epoch 30 | Loss: 0.2632

Приклади генерації
1 Temp 0.5: лис микита біг микита біг місце місце біг місце біг біг місце фельдшериці біг місце фельдшериці біг місце фельдшериці біг місце фельдшериці біг місце фельдшериці біг місце фельдшериці біг місце фельдшериці біг місце фельдшериці біг місце фельдшериці біг місце фельдшериці біг місце фельдшериці біг місце фельдшериці біг місце фельдшериці біг місце фельдшериці біг
2 Temp 0.7: цар лев цар лев цар цар цар цар цар цар цар цар цар й цар цар цар, то я не раз журився! ой мій зайку тут наплели. ну ж твої слова так сподобався попові мало і шахрай та гидкі ходить перед входом під стіною притуливсь? як пишавсь з
3 Temp 1.0: ті безхвості ті став знаменита мужики наказ хвоста урвав княгиня аби бідний батько призьбі хрюка забутий. ну, а як чкурне серед поля! і шахрай баб